# **Measuring Bias in Regression Tasks**


In order to make sure that your algorithm treats everyone equally, it is important to measure the bias. Here, we will illustrate how to measure bias in a regression context. 

Suppose we have a dataset involving grades of students from a school, and a range of features that could be useful for prediction of their grades. If some of these features involve sex, the models trained from this dataset could show bias against a certain sex. We would like to avoid such discrimination, because it causes prejudice and might cause disadvantages to some people.

In this notebook, we will:
1. Install and import useful modules and load the data
2. Code Bias Metrics for regression tasks
3. Download student performance dataset
4. Train a regression model to predict grades
5. Check this model for bias

We will implement a Light Gradient Boosting Machine with [scikit-learn](https://scikit-learn.org/stable/) in 4, and check it's bias in 5.



## **Base Modules**

In [ ]:
!pip install scikit-learn==0.21.3
import pandas as pd
import numpy as np
%matplotlib inline

## **Bias Metrics**

Below we display the code for a number of different metrics useful in quantifying bias in regression tasks.

In [ ]:
def DispImpactQ90(yobs, ypred, gmaj, gmin):    
    ### Get quantile 90%
    QX = np.quantile(ypred, .9)
    
    ### Disparate Impact (a.k.a. Adverse Impact Ratio)
    SR_min = (gmin * (ypred > QX)).sum()/gmin.sum() # success rate minority
    SR_maj = (gmaj * (ypred > QX)).sum()/gmaj.sum() # success rate majority
    
    return SR_min/SR_maj

def DispImpactQ80(yobs, ypred, gmaj, gmin):   
    ### Get quantile 80%
    QX = np.quantile(ypred, .8)
    
    ### Disparate Impact (a.k.a. Adverse Impact Ratio)
    SR_min = (gmin * (ypred > QX)).sum()/gmin.sum() # success rate minority
    SR_maj = (gmaj * (ypred > QX)).sum()/gmaj.sum() # success rate majority
    
    return SR_min/SR_maj

def DispImpactQ50(yobs, ypred, gmaj, gmin):  
    ### Get quantile 50%
    QX = np.quantile(ypred, .5)
    
    ### Disparate Impact (a.k.a. Adverse Impact Ratio)
    SR_min = (gmin * (ypred > QX)).sum()/gmin.sum() # success rate minority
    SR_maj = (gmaj * (ypred > QX)).sum()/gmaj.sum() # success rate majority
    
    return SR_min/SR_maj

def NoAdvImpactLevel(yobs, ypred, gmaj, gmin):  
    ##### find maximum score that not allow adverse impact
    ### grid
    q = np.linspace(1.0, 0.0, 100)
    pred_value = []
    sr_maj = []
    sr_min = []
    
    ### try different values
    for (i, v) in enumerate(q):
        pred = np.quantile(ypred, v)
        pass_members = (ypred > pred)
        if ( sum(gmin * pass_members)/sum(gmin) ) / ( sum(gmaj * pass_members)/sum(gmaj) ) > 0.8:
            if ( sum(gmin * pass_members)/sum(gmin) ) / ( sum(gmaj * pass_members)/sum(gmaj) ) < 1.2:
                break
            
    return pred

def AvgScoreSpread(yobs, ypred, gmaj, gmin):
    ### get averages
    avg_min = ypred[gmin==1].mean()
    avg_maj = ypred[gmaj==1].mean()
    
    return avg_min - avg_maj

def AvgScoreSpreadTop20(yobs, ypred, gmaj, gmin):    
    ### Get top 20%
    topX = np.argsort(ypred)[-int(ypred.shape[0]*0.2):]
    ypred = ypred[topX]
    gmin = gmin[topX]
    gmaj = gmaj[topX]
    
    ### get averages
    avg_min = ypred[gmin==1].mean()
    avg_maj = ypred[gmaj==1].mean()
    
    return avg_min - avg_maj

    
def ZScoreSpread(yobs, ypred, gmaj, gmin):
    ### get means and pooled std
    avg_min = ypred[gmin==1].mean()  
    avg_maj = ypred[gmaj==1].mean()
    n_min = sum(gmin==1)
    n_maj = sum(gmaj==1)
    pool_sd = np.sqrt( (n_min-1) * (ypred[gmin==1].std() ** 2.0 + (n_maj-1) * ypred[gmaj==1].std() ** 2.0) / (n_maj + n_min - 2) )
    
    return (avg_min - avg_maj)/pool_sd

def ZScoreSpreadTop20(yobs, ypred, gmaj, gmin):
    ### Get top 20%
    topX = np.argsort(ypred)[-int(ypred.shape[0]*0.2):]
    ypred = ypred[topX]
    gmin = gmin[topX]
    gmaj = gmaj[topX]
    
    ### get means and pooled std
    avg_min = ypred[gmin==1].mean()  
    avg_maj = ypred[gmaj==1].mean()
    n_min = sum(gmin==1)
    n_maj = sum(gmaj==1)
    pool_sd = np.sqrt( (n_min-1) * (ypred[gmin==1].std() ** 2.0 + (n_maj-1) * ypred[gmaj==1].std() ** 2.0) / (n_maj + n_min - 2) )
    
    return (avg_min - avg_maj)/pool_sd

def AdvImpactAUC(yobs, ypred, gmaj, gmin):
    ### thresholds
    thresh = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]

    ### pass rate per thresh
    pass_min = []
    pass_maj = []
    for (i, t) in enumerate(thresh):
        pass_value = np.quantile(ypred, t)
        pass_min += [(gmin[ypred > pass_value]==1).sum() / (gmin==1).sum()]
        pass_maj += [(gmaj[ypred > pass_value]==1).sum() / (gmaj==1).sum()]

    ### AUC
    return sum( (np.array(pass_min[1:])-np.array(pass_min[:-1]))*(np.array(pass_maj[1:])-np.array(pass_maj[0])) )

def ConcVal(yobs, ypred):
    return np.corrcoef(ypred, yobs)[1,0]

def RMSE(yobs, ypred):
    return np.sqrt( ((yobs - ypred) ** 2.0).mean() )

def ConcValSpread(yobs, ypred, gmaj, gmin):
    cv_min = np.corrcoef(ypred[gmin==1], yobs[gmin==1])[1,0]
    cv_maj = np.corrcoef(ypred[gmaj==1], yobs[gmaj==1])[1,0]
    return cv_min - cv_maj

def RmseRatio(yobs, ypred, gmaj, gmin):
    rmse_min = np.sqrt( ( (yobs[gmin==1] - ypred[gmin==1]) ** 2.0 ).mean() )
    rmse_maj = np.sqrt( ( (yobs[gmaj==1] - ypred[gmaj==1]) ** 2.0).mean() )
    return rmse_min / rmse_maj

def ConcValSpreadTop20(yobs, ypred, gmaj, gmin):
    ### Get top 20%
    topX = np.argsort(ypred)[-int(ypred.shape[0]*0.2):]
    ypred = ypred[topX]
    yobs = yobs[topX]
    gmin = gmin[topX]
    gmaj = gmaj[topX]
    
    ### compute metric
    cv_min = np.corrcoef(ypred[gmin==1], yobs[gmin==1])[1,0]
    cv_maj = np.corrcoef(ypred[gmaj==1], yobs[gmaj==1])[1,0]
    return cv_min - cv_maj


def RmseRatioSpreadTop20(yobs, ypred, gmaj, gmin):
    ### Get top 20%
    topX = np.argsort(ypred)[-int(ypred.shape[0]*0.2):]
    ypred = ypred[topX]
    yobs = yobs[topX]
    gmin = gmin[topX]
    gmaj = gmaj[topX]
    
    ### compute metric
    rmse_min = np.sqrt( ( (yobs[gmin==1] - ypred[gmin==1]) ** 2.0).mean() )
    rmse_maj = np.sqrt( ( (yobs[gmaj==1] - ypred[gmaj==1]) ** 2.0).mean() )
    return rmse_min / rmse_maj

def AI_ROC(yobs, ypred, gmaj, gmin):
    from matplotlib import pyplot as plt
    ### thresholds
    thresh = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]

    ### pass rate per thresh
    pass_min = []
    pass_maj = []
    for (i, t) in enumerate(thresh):
        pass_value = np.quantile(ypred, t)
        pass_min += [(gmin[ypred > pass_value]==1).sum() / (gmin==1).sum()]
        pass_maj += [(gmaj[ypred > pass_value]==1).sum() / (gmaj==1).sum()]

    # charting
    fig, ax = plt.subplots()
    ax.plot(np.array(pass_min) * 100.0, np.array(pass_maj) * 100.0)
    ax.plot([0, 100], [0, 100], linestyle='--')
    ax.set_xlabel("Minority Pass Rate %")
    ax.set_ylabel("Majority Pass Rate %")
    return plt.show()


def AI_Curve(yobs, ypred, gmaj, gmin):
    from matplotlib import pyplot as plt
    # quantiles
    q = np.linspace(0.0, 1.0, 200)
    pred_value = []
    other_form_maj = []
    other_form_min = []
    
    # grid values
    for (i, v) in enumerate(q):
        pred = np.quantile(ypred, v)
        pass_members = (ypred > pred)
        other_form_maj += [sum(gmaj * pass_members)/sum(gmaj)]
        other_form_min += [sum(gmin * pass_members)/sum(gmin)]
        pred_value += [pred]

    # charting
    fig, ax = plt.subplots()
    ax.plot(pred_value, np.array(other_form_min)/np.array(other_form_maj))
    ax.plot([np.quantile(ypred, 0.0), np.quantile(ypred, 1.0)], [0.8, 0.8], linestyle='--')
    ax.set_xlabel("Score")
    ax.set_ylabel("Adverse Impact")
    return plt.show()


def compute_model_metrics(ypred, yobs=None, gmaj=None, gmin=None):
    
    if yobs is None:
        # metrics
        perf_metrics = {}
        # fairness metrics
        fair_metrics = {"Disparate Impact (Q 90%)": DispImpactQ90,
                        "Disparate Impact (Q 80%)": DispImpactQ80,
                        "Disparate Impact (Q 50%)": DispImpactQ50,
                        "Avg Score Spread": AvgScoreSpread,
                        "Avg Score Spread (top 20%)": AvgScoreSpreadTop20,
                        "Z-score Spread": ZScoreSpread,
                        "Z-score Spread (top 20%)": ZScoreSpreadTop20,
                        "Adv Impact AUC": AdvImpactAUC
                       }
        
    else:
        # metrics
        from sklearn import metrics
        perf_metrics = {"Concurrent Validity": ConcVal, 
                        "RMSE": RMSE, 
                       }
        # fairness metrics
        fair_metrics = {"Disparate Impact (Q 90%)": DispImpactQ90,
                        "Disparate Impact (Q 80%)": DispImpactQ80,
                        "Disparate Impact (Q 50%)": DispImpactQ50,
                        "Avg Score Spread": AvgScoreSpread,
                        "Avg Score Spread (top 20%)": AvgScoreSpreadTop20,
                        "Z-score Spread": ZScoreSpread,
                        "Z-score Spread (top 20%)": ZScoreSpreadTop20,
                        "Adv Impact AUC": AdvImpactAUC,
                        "Concurrent Validity Spread": ConcValSpread,
                        "RMSE Ratio": RmseRatio,
                        "Concurrent Validity Spread (top 20%)": ConcValSpreadTop20,
                        "RMSE Ratio (top 20%)": RmseRatioSpreadTop20
                       }
        
    # compute performance metrics
    metrics = []
    for pf in perf_metrics.keys():
        metrics += [[pf, perf_metrics[pf](yobs, ypred)]]
            
    if (gmaj is not None) and (gmin is not None):
        for ff in fair_metrics.keys():
            metrics += [[ff, fair_metrics[ff](yobs, ypred, gmaj, gmin)]]
    
    return pd.DataFrame(metrics, columns=["Metric", "Value"])


## **Load Data**

If running in colab the dataset is obtained in the next cell with the !wget command. If running locally, download zip file from (https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip) and unpack the dataset in the same folder as notebook.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
!unzip student.zip

--2022-04-26 18:21:39--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip.1’

student.zip.1       100%[===================>]  20.00K  --.-KB/s    in 0.05s   

2022-04-26 18:21:39 (404 KB/s) - ‘student.zip.1’ saved [20478/20478]

Archive:  student.zip
replace student-mat.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [ ]:
# read dataset with pandas
data = pd.read_csv('student-mat.csv',delimiter=';')
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [ ]:
# check data column types
data.dtypes

school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
G1             int64
G2             int64
G3             int64
dtype: object

## **Preprocess Data and Train Model**

In the following section we will
1. Drop the grades from years 1 and 2 because of high correlation with year 3
2. Separate categorical and numerical columns
3. Create a simple preprocessing Pipeline with scikit-learn
4. Train a LGBM model with scikit-learn


In [ ]:
# drop grades from first and second year because high correlation with third

data.drop(columns = ['G1','G2'],inplace=True)

In [ ]:
# separate categorical and numerical data

categorical = []
numerical = []

for col in data.columns:
  if col == 'G3':
    pass
  elif col == 'sex':
    pass
  elif data[col].dtype == object:
    categorical.append(col)
  else:
    numerical.append(col)

print (categorical)
print (numerical)


['school', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


Below we create a simple pipeline that will one hot encode categorical columns, and standard scale numerical ones.

In [ ]:
# create a pipeline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)
    ])

Below we train test split our data with a 70/30 ratio to prepare for training. We also remove the 'sex' attribute because we do not want to use protected attributes in training.

In [ ]:
# train test split 70/30

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.3,random_state=42)

df_train_X = df_train.drop(columns=['G3','sex'])
df_test_X = df_test.drop(columns=['G3','sex'])
df_train_y = df_train['G3']
df_test_y = df_test['G3']

We train a LGBM regression model. It performed best of 5 regression models tested.

In [ ]:
# fit and predict.

from lightgbm import LGBMRegressor

lgbmr = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', LGBMRegressor())])

lgbmr.fit(df_train_X,df_train_y)
y_pred = lgbmr.predict(df_test_X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_int = np.zeros((n_samples, n_features), dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details an

## **Display Performance and Bias Metrics**

We must define vectors containing the sex of the individuals to check for bias.

In [ ]:
# define minority and majority vectors
gmaj = df_test['sex']=='M'
gmin = df_test['sex']=='F'
gmaj = gmaj.to_numpy()
gmin = gmin.to_numpy()
yobs = df_test_y.to_numpy()

We can now compute a range of accuracy and fairness metrics, displayed in the dataframe below. The first two are accuracy metrics, the remaining 11 are fairness metrics.

In [ ]:
compute_model_metrics(ypred=y_pred, yobs=yobs, gmaj=gmaj, gmin=gmin)

,Metric,Value
0,Concurrent Validity,0.537991
1,RMSE,3.973615
2,Disparate Impact (Q 90%),0.751232
3,Disparate Impact (Q 80%),0.525862
4,Disparate Impact (Q 50%),0.949944
5,Avg Score Spread,-0.706965
6,Avg Score Spread (top 20%),0.240590
7,Z-score Spread,-0.050052
8,Z-score Spread (top 20%),0.115542
9,Adv Impact AUC,0.579141


## ***Final Remarks***

We observe through some metrics that there is a strong negative bias towards students of female sex. For instance we look at the Disparate Impact Q 80% of 0.52 (a value of 1 would be fair). This means male students are about 2 times more likely to be predicted in the top 20% of grades. We would like to avoid such prediction bias, as it might be detrimental to female students in admissions or other scenarios.